In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# data cleaning
def dataClean(df):
  # there's an extra column at the end of the data set that needs removing.
  df = df.drop(columns=["id", "Unnamed: 32"])
  # other than this, there is no more data cleaning to do.
  return df

In [ ]:
# First we isolate the classification column of the datasset ('diagnosis').
# Because 'diagnosis' uses M & B as labels and CFS uses numbers, we map
# M and B to 1 and 0, respectively.
def predClassMapping(df):
  df['diagnosis'] = pd.Series(df.diagnosis).map({'M':1,'B':0});
  return df

In [ ]:
#12/24
# CORRELATION FEATURE SELECTION
R_XY = None
# tauRed = 0.8, k = 6
def corrFeatureSelection(df, k = 10, tauRedundancy = 0.8):
  # 1) Sort features by absolute correlation with the label (descending)
  targetCorr = df.corr()['diagnosis'].abs().sort_values(ascending=False)
  R_XY = targetCorr
  # 2) Now pick features one by one from the most strongly correlated
  #    to the least, but skip any feature that is "too correlated"

  selectedFeatures = []
  rejectedFeatures = []

  for feature in targetCorr.index:
    if feature == 'diagnosis':
      continue  # Skip the label itself

    # Check correlation with already selected features
    aboveThreshold = False
    for alreadySelected in selectedFeatures:
      # If the correlation is above the threshold, skip
      if abs(df[feature].corr(df[alreadySelected])) > tauRedundancy:
        aboveThreshold = True
        rejectedFeatures.append(feature)
        break

    if not aboveThreshold:
      selectedFeatures.append(feature)

    # If we already have our 10 features, stop
    if len(selectedFeatures) == k:
      break

  print("Selected features:", selectedFeatures)
  print("Num features:", len(selectedFeatures))
  # This will give up to 10 features that are:
  # - highly correlated with the label (because we started with that sorted list),
  # - but have low correlation with each other (due to our threshold check).

  selectedDF = df[['diagnosis'] + selectedFeatures]

  return { "sfCorrMatrix": selectedDF.corr(), "selectedDF": selectedDF,"selectedFeatures": selectedFeatures, "rejectedFeatures": rejectedFeatures }

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Load and process data
df = pd.read_csv("data.csv")
df = dataClean(df)
df = predClassMapping(df)

# Compute correlations and perform CFS
R_XY = df.corr()['diagnosis'].abs().sort_values(ascending=False)
# Remove the 'diagnosis' entry from the correlation Series
R_XY = R_XY.drop('diagnosis')

res = corrFeatureSelection(df, k=6, tauRedundancy=0.8)

# Extract features and their correlation values (without 'diagnosis')
features = R_XY.index
correlations = R_XY.values

# Create a color list for each feature based on its status:
colors = []
for feature in features:
    if feature in res["selectedFeatures"]:
        colors.append("green")
    elif feature in res["rejectedFeatures"]:
        colors.append("red")
    else:
        # Implicitly rejected (not processed) features
        colors.append("blue")

# Plot the bar chart with the custom colors
plt.figure(figsize=(12, 6))
plt.bar(features, correlations, color=colors)
plt.title('Relevancy of each feature', pad=20)
plt.ylabel('Pearson Correlation Coefficient (0 to 1)')
plt.xticks(rotation=90)

# Create a custom legend to clarify the colors
selected_patch = mpatches.Patch(color="green", label="Selected Features")
rejected_patch = mpatches.Patch(color="red", label="Rejected Features")
implicit_patch = mpatches.Patch(color="blue", label="Implicitly Rejected Features")
plt.legend(handles=[selected_patch, rejected_patch, implicit_patch])

plt.show()

In [ ]:
# 10/24
# SPLIT DATA SET
import sklearn as skl
from sklearn.model_selection import train_test_split


def splitData(df, testSize = 0.20):
  # Split the label column from the features
  Y = df.loc[:, 'diagnosis']
  X = df.loc[:, df.columns != 'diagnosis']

  # Now split the X and Y datasets into train/test (0.8/0.2 split by default)
  Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=testSize, random_state=23)
  return [Xtrain, Xtest, Ytrain, Ytest]

In [ ]:
# Plot all the features in a given dataframe set. 
# 26/02/2025
import math

def plot_all_features(df):
  # Select only numeric columns (ignoring non-numeric ones like 'diagnosis')
  numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
  print("Numeric columns to plot:", numeric_cols)
  
  # Determine subplot grid size: 3 columns per row
  num_plots = len(numeric_cols)
  ncols = 3
  nrows = math.ceil(num_plots / ncols)
  
  fig, axes = plt.subplots(nrows, ncols, figsize=(5*ncols, 4*nrows), squeeze=False)
  axes = axes.flatten()
  
  # Create a line plot for each numeric feature
  for i, col in enumerate(numeric_cols):
    axes[i].plot(df.index, df[col], marker='o', linestyle='-')
    axes[i].set_title(col)
    axes[i].set_xlabel('Index')
    axes[i].set_ylabel(col)
    axes[i].grid(True)
  
  # Remove any empty subplots if grid has extra axes
  for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])
  
  plt.tight_layout()
  plt.show()
  print(type(df.iloc[0]))
  return df


In [ ]:
# ADDITIVE SMOOTHING
#26/02/2025

def additiveSmoothing(x_i: np.ndarray, alpha: float) -> np.ndarray:
  """
  x_i   - a NumPy array where the first element is a diagnosis (or label)
          and the remaining elements are numeric counts.
  alpha - the pseudocount (> 0; if alpha is 0, no smoothing is applied)
  
  The smoothed value for each numeric element (x_j, for j >= 1) is computed as:
    x_j_new = N * ((x_j + alpha) / (N + alpha * d))
  where:
    N = sum of numeric counts (x_i[1:]),
    d = number of numeric features (length of x_i[1:]).
    
  The first element (diagnosis) is left unchanged.
  """

  # Work on a copy to avoid modifying the original array
  x_iCopy = x_i.copy()
  
  # Extract the diagnosis (first element)
  diagnosis = x_iCopy[0]
  
  # Extract the numeric part, converting to float if necessary
  x_iCopy = x_iCopy[1:].astype(float)
  
  # Calculate the total count (N) and the number of features (d)
  N = x_iCopy.sum()
  d = len(x_iCopy)
  
  # Compute the smoothed numeric values using vectorized operations
  x_iNew = N * (x_iCopy + alpha) / (N + alpha * d)
  
  # Combine the unchanged diagnosis with the smoothed values
  result = np.concatenate(([diagnosis], x_iNew))
  return result

def dataSmooth(df, alpha):
  for i in range(len(df)):
    x_i = df.iloc[i].to_numpy()
    x_iNew = additiveSmoothing(x_i, alpha)
    df.iloc[i] = x_iNew
  
  return df


In [ ]:
# 26/02/2025
def smoothTestFunc():
  # Load the CSV file
  df = pd.read_csv("data.csv")
  df = dataClean(df)
  df = predClassMapping(df)

  plot_all_features(df)
  print("==================== AFTER SMOOTHING================================")
  for i in range(len(df)):
    x_i = df.iloc[i].to_numpy()
    x_iNew = additiveSmoothing(x_i, 100000)
    df.iloc[i] = x_iNew

  plot_all_features(df)

In [ ]:
# 01/02/2025
# This is the data preprocessing handler. Can be used to execute any one or multiple forms of data preprocessing (cleaning, mapping, feature selection, data splitting).
# NOTE: If data set splitting and CFS are to be done in the same call, the CFS option MUST be first in the `processes` array (so that the dataset is feature selected, and THEN split)
def dataPreProcessing(dsFile = "data", df = None, processes = ["clean", "predMap", "CFS", "splitSet"], kFeatures = 10, tauRedundancy = 0.8, testSize = 0.20):
  returnVars = {"df": None, "CFS Corr Matrix": None, "Xtrain": None, "Xtest": None, "Ytrain": None, "Ytest": None}
  if df is None:
    df = pd.read_csv(dsFile + '.csv')

  for process in processes:
    match process:
      case "clean":
        df = dataClean(df)
        df = additiveSmoothing(df, 100000)
      case "predMap":
        df = predClassMapping(df)
      case "CFS":
        res = corrFeatureSelection(df, kFeatures, tauRedundancy)
        returnVars["CFS Corr Matrix"] = res["sfCorrMatrix"]
        df = res["selectedDF"]
      case "splitSet":
        Xtrain, Xtest, Ytrain, Ytest = splitData(df, testSize)
        returnVars["Xtrain"] = Xtrain
        returnVars["Xtest"] = Xtest
        returnVars["Ytrain"] = Ytrain
        returnVars["Ytest"] = Ytest
      case _:
        raise Exception("Processes param is empty. Options: ['clean', 'predMap', 'CFS', 'splitSet']")

  returnVars["df"] = df
  return returnVars